In [1]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""

import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib

import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat

debug=False

In [2]:
# Mosaic multiple files
mosaic_ts_info = False
from rasterio.merge import merge

# ====== MERGE CALIBRATED GLACIERS INTO DEGREE PIXELS ======
# Export all .tifs to a common projection that will enable merging them smoothly
dst_crs = 'EPSG:4326'

if mosaic_ts_info:
    
    # All Ts filenames
    ts_fullfns = []
    ts_year_fullfns = []
    ts_doy_fullfns = []
    ts_dayfrac_fullfns = []
    for i in os.listdir(debris_prms.ts_fp):
        if i.endswith('.tif'):
            if 'tsurfC' in i:
                ts_fullfns.append(debris_prms.ts_fp + i)
            elif 'year' in i:
                ts_year_fullfns.append(debris_prms.ts_fp + i)
            elif 'doy' in i:
                ts_doy_fullfns.append(debris_prms.ts_fp + i)
            elif 'dayfrac' in i:
                ts_dayfrac_fullfns.append(debris_prms.ts_fp + i)

    ts_fullfns = sorted(ts_fullfns)
    ts_year_fullfns = sorted(ts_year_fullfns)
    ts_doy_fullfns = sorted(ts_doy_fullfns)
    ts_dayfrac_fullfns = sorted(ts_dayfrac_fullfns)
    
    if debug:
        print(ts_fullfns)
        print(ts_year_fullfns)
        print(ts_doy_fullfns)
        print(ts_dayfrac_fullfns)

    def mosaic_fullfns(list_fullfns, fn_merged=None):
        if fn_merged is None:
            fn_merged = list_fullfns[0].split('/')[-1].split('-')[0] + '.tif'
        src_files_to_mosaic = []
        for fn in list_fullfns:
            with rasterio.open(fn) as src:
                src = rasterio.open(fn)
                src_files_to_mosaic.append(src)

        mos, out_trans = merge(src_files_to_mosaic)

        with rasterio.open(debris_prms.ts_fp + fn_merged,"w", driver ='Gtiff',count=1,
                           height= mos.shape[1],
                           width= mos.shape[2],
                           transform= out_trans,
                           crs= src.crs,
                           dtype= src.dtypes[0]) as dest:
            dest.write(mos)
            
    mosaic_fullfns(ts_fullfns)
    mosaic_fullfns(ts_year_fullfns)
    mosaic_fullfns(ts_doy_fullfns)
    mosaic_fullfns(ts_dayfrac_fullfns)

In [3]:
ts_fullfns = debris_prms.ts_fullfns_dict[debris_prms.roi]
ts_year_fullfns = debris_prms.ts_year_fullfns_dict[debris_prms.roi]
ts_doy_fullfns = debris_prms.ts_doy_fullfns_dict[debris_prms.roi]
ts_dayfrac_fullfns = debris_prms.ts_dayfrac_fullfns_dict[debris_prms.roi]

In [ ]:
add_ts_fns = True
if add_ts_fns:
    
    # Load all debris-covered glaciers
    dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
    dc_shp = dc_shp.sort_values(by=['RGIId'])
    dc_shp.reset_index(inplace=True, drop=True)
    
    dc_rgiids = [x.split('-')[1] for x in dc_shp.RGIId.values]
    print(len(dc_rgiids), 'ex:', dc_rgiids[0],'\n')
    main_glac_rgi_all = debris_prms.selectglaciersrgitable(dc_rgiids)

    # Ts filenames dataframe
    if os.path.exists(debris_prms.ts_fp + debris_prms.ts_fns_fn):
        ts_fns_df = pd.read_csv(debris_prms.ts_fp + debris_prms.ts_fns_fn)
    else:
        ts_fns_cns = ['RGIId', 'ts_fullfn', 'ts_year_fullfn', 'ts_doy_fullfn', 'ts_dayfrac_fullfn']
        ts_fns_df = pd.DataFrame(np.zeros((main_glac_rgi_all.shape[0], len(ts_fns_cns))), columns=ts_fns_cns)
        ts_fns_df['RGIId'] = main_glac_rgi_all['RGIId']
        
    if debug:
        print(ts_fullfns)
        print(ts_year_fullfns)
        print(ts_doy_fullfns)
        print(ts_dayfrac_fullfns)
        
    ts_fn_wglacier = None
    ts_year_fn_wglacier = None
    ts_doy_fn_wglacier = None
    ts_dayfrac_fn_wglacier = None
    
    # ===== PROCESS EACH GLACIER =====
    for nglac, glac_idx in enumerate(main_glac_rgi_all.index.values):
#     for nglac, glac_idx in enumerate([main_glac_rgi_all.index.values[0]]):
        glac_str = main_glac_rgi_all.loc[glac_idx,'rgino_str']
        rgiid = main_glac_rgi_all.loc[glac_idx,'RGIId']
        region = glac_str.split('.')[0]

        if int(region) < 10:
            glac_str_noleadzero = str(int(glac_str.split('.')[0])) + '.' + glac_str.split('.')[1]
        else:
            glac_str_noleadzero = glac_str

        print(nglac, glac_idx, rgiid)

        # Create glacier feature from ice thickness raster
        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'

        if os.path.exists(thick_dir + thick_fn):
#         if os.path.exists(thick_dir + thick_fn) and '0.0' in ts_fns_df.loc[glac_idx,:].values:
#             print(nglac, glac_idx, rgiid)

            gf = create_glacfeat(thick_dir, thick_fn)

            # Debris shape layer processing
            dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                              str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
            if os.path.exists(dc_shp_proj_fn) == False:
                dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
                dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                dc_shp_single = dc_shp_single.reset_index()
                dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                dc_shp_proj.to_file(dc_shp_proj_fn)
            dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
            dc_shp_lyr = dc_shp_ds.GetLayer()

            # ==== CHECK IF TIF HAS SURFACE TEMPERATURE DATA OVER THE GLACIER =====
            # Add the filenames
            fn_dict = OrderedDict()
            # DEM
            z1_fp = debris_prms.oggm_fp + 'dems/RGI60-' + str(region.zfill(2)) + '/'
            z1_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_dem.tif'
            fn_dict['z1'] = z1_fp + z1_fn
            # Ice thickness
            thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
            thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
            fn_dict['ice_thick'] = thick_dir + thick_fn
            # Ts
            ts_dict_keys = []
            for nfn, ts_fn in enumerate(ts_fullfns):
                fn_dict['ts_' + str(nfn)] = ts_fn
                ts_dict_keys.append('ts_' + str(nfn))
            ts_year_dict_keys = []
            for nfn, ts_year_fn in enumerate(ts_year_fullfns):
                fn_dict['ts_year_' + str(nfn)] = ts_year_fn
                ts_year_dict_keys.append('ts_year_' + str(nfn))
            ts_doy_dict_keys = []
            for nfn, ts_doy_fn in enumerate(ts_doy_fullfns):
                fn_dict['ts_doy_' + str(nfn)] = ts_doy_fn
                ts_doy_dict_keys.append('ts_doy_' + str(nfn))
            ts_dayfrac_dict_keys = []  
            for nfn, ts_dayfrac_fn in enumerate(ts_dayfrac_fullfns):
                fn_dict['ts_dayfrac_' + str(nfn)] = ts_dayfrac_fn
                ts_dayfrac_dict_keys.append('ts_dayfrac_' + str(nfn))

            # ===== PROCESS THE DATA =====
            #Expand extent to include buffered region around glacier polygon
            warp_extent = geolib.pad_extent(gf.glac_geom_extent, width=debris_prms.buff_dist)
            #Warp everything to common res/extent/proj
            z1_gt = gdal.Open(fn_dict['z1']).GetGeoTransform()
            z1_res = np.min([z1_gt[1], -z1_gt[5]])
            ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, extent=warp_extent, 
                                               t_srs=gf.aea_srs, verbose=False, r='cubic')
            ds_dict = dict(zip(fn_dict.keys(), ds_list))
            gf.ds_dict = ds_dict

            if 'z1' in ds_dict:
                #This is False over glacier polygon surface, True elsewhere - can be applied directly
                glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1'])
                glac_geom_mask_copy = glac_geom_mask.copy()
                gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)
                # Debris cover
                dc_shp_lyr_mask = geolib.lyr2mask(dc_shp_lyr, ds_dict['ice_thick'])
                gf.dc_mask = np.ma.mask_or(dc_shp_lyr_mask, glac_geom_mask)
                gf.dc_area = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=gf.dc_mask)

                # ====== SURFACE TEMPERATURE =====
                find_ts = True
                ts_key_count = 0
                if debug:
                    print('\nSURFACE TEMPERATURE')
                while find_ts and ts_key_count < len(ts_dict_keys):
                    ts_key = ts_dict_keys[ts_key_count]
                    if debug:
                        print(ts_key, ts_fullfns[ts_key_count].split('/')[-1])
                    gf.ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.ts.mask = np.ma.mask_or(
                        glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.ts.data, np.isnan(gf.ts.data))))
                    gf.dc_ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.dc_ts.mask = gf.dc_mask
                    # Count pixels
                    dc_ts_pixels = len(gf.ts.nonzero()[0])
                    if dc_ts_pixels / gf.dc_area.count() * 100 > 75:
                        find_ts = False
                        # Export raster
                        ts_fn = rgiid + '_ts.tif'
                        ts_fp = debris_prms.oggm_ts_fp + 'RGI60-' + debris_prms.roi + '/'
                        if not os.path.exists(ts_fp):
                            os.makedirs(ts_fp)
                        iolib.writeGTiff(gf.ts, ts_fp + ts_fn, gf.ds_dict['z1'])
                        ts_fn_wglacier = ts_fp + ts_fn
                        
                        if debug:
                            print('\n# z1 dc pixels:', gf.dc_area.count())
                            print('# dc_ts_pixels:', dc_ts_pixels)
                            print('ts_fn:', ts_fn_wglacier)
                            var_full2plot = gf.ts.copy()
                            clim = malib.calcperc(var_full2plot, (2,98))
                            plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'Ts (degC)', 
                                       close_fig=False)
                    ts_key_count += 1
                    
                # ====== TS YEAR =====
                find_ts_year = True
                ts_key_count = 0
                if debug:
                    print('\nYEAR')
                while find_ts_year and ts_key_count < len(ts_year_dict_keys):
                    ts_key = ts_year_dict_keys[ts_key_count]
                    if debug:
                        print(ts_key, ts_year_fullfns[ts_key_count].split('/')[-1])
                    gf.ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.ts.mask = np.ma.mask_or(
                        glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.ts.data, np.isnan(gf.ts.data))))
                    gf.dc_ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.dc_ts.mask = gf.dc_mask
                    # Count Ts pixels
                    dc_ts_pixels = len(gf.ts.nonzero()[0])
                    if dc_ts_pixels / gf.dc_area.count() * 100 > 75:
                        find_ts_year = False
                        # Export raster
                        ts_year_fn = rgiid + '_ts_year.tif'
                        ts_year_fp = debris_prms.oggm_ts_info_fp + 'RGI60-' + debris_prms.roi + '/'
                        if not os.path.exists(ts_year_fp):
                            os.makedirs(ts_year_fp)
                        iolib.writeGTiff(gf.ts, ts_year_fp + ts_year_fn, gf.ds_dict['z1'])
                        ts_year_fn_wglacier = ts_year_fp + ts_year_fn
                        
                        if debug:
                            print('\n# z1 dc pixels:', gf.dc_area.count())
                            print('# dc_ts_pixels:', dc_ts_pixels)
                            print('ts_year_fn:', ts_year_fn_wglacier)
                            var_full2plot = gf.ts.copy()
                            clim = (2013,2018)
                            plot_array(var_full2plot, clim, [glac_str + ' Ts_year'], 'inferno', 'Year', 
                                       close_fig=False)
                    ts_key_count += 1
                    
                # ====== TS DOY =====
                find_ts_doy = True
                ts_key_count = 0
                if debug:
                    print('\nDAY OF YEAR')
                while find_ts_doy and ts_key_count < len(ts_doy_dict_keys):
                    ts_key = ts_doy_dict_keys[ts_key_count]
                    if debug:
                        print(ts_key, ts_doy_fullfns[ts_key_count].split('/')[-1])
                    gf.ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.ts.mask = np.ma.mask_or(
                        glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.ts.data, np.isnan(gf.ts.data))))
                    gf.dc_ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.dc_ts.mask = gf.dc_mask
                    # Count Ts pixels
                    dc_ts_pixels = len(gf.ts.nonzero()[0])
                    if dc_ts_pixels / gf.dc_area.count() * 100 > 75:
                        find_ts_doy = False
                        # Export raster
                        ts_doy_fn = rgiid + '_ts_doy.tif'
                        ts_doy_fp = debris_prms.oggm_ts_info_fp + 'RGI60-' + debris_prms.roi + '/'
                        if not os.path.exists(ts_doy_fp):
                            os.makedirs(ts_doy_fp)
                        iolib.writeGTiff(gf.ts, ts_doy_fp + ts_doy_fn, gf.ds_dict['z1'])
                        ts_doy_fn_wglacier = ts_doy_fp + ts_doy_fn
                        if debug:
                            print('\n# z1 dc pixels:', gf.dc_area.count())
                            print('# dc_ts_pixels:', dc_ts_pixels)
                            print('ts_doy_fn:', ts_doy_fn_wglacier)
                            var_full2plot = gf.ts.copy()
                            clim = (160,280)
                            plot_array(var_full2plot, clim, [glac_str + ' Ts_doy'], 'inferno', 'DOY', 
                                       close_fig=False)
                    ts_key_count += 1
                    
                # ====== TS DAYFRAC =====
                find_ts_dayfrac = True
                ts_key_count = 0
                if debug:
                    print('\nDAY FRACTION')
                while find_ts_dayfrac and ts_key_count < len(ts_dayfrac_dict_keys):
                    ts_key = ts_dayfrac_dict_keys[ts_key_count]
                    if debug:
                        print(ts_key, ts_dayfrac_fullfns[ts_key_count].split('/')[-1])
                    gf.ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.ts.mask = np.ma.mask_or(
                        glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.ts.data, np.isnan(gf.ts.data))))
                    gf.dc_ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.dc_ts.mask = gf.dc_mask
                    # Count Ts pixels
                    dc_ts_pixels = len(gf.ts.nonzero()[0])
                    if dc_ts_pixels / gf.dc_area.count() * 100 > 75:
                        find_ts_dayfrac = False
                        # Export raster
                        ts_dayfrac_fn = rgiid + '_ts_dayfrac.tif'
                        ts_dayfrac_fp = debris_prms.oggm_ts_info_fp + 'RGI60-' + debris_prms.roi + '/'
                        if not os.path.exists(ts_dayfrac_fp):
                            os.makedirs(ts_dayfrac_fp)
                        iolib.writeGTiff(gf.ts, ts_dayfrac_fp + ts_dayfrac_fn, gf.ds_dict['z1'])
                        ts_dayfrac_fn_wglacier = ts_dayfrac_fp + ts_dayfrac_fn
                        if debug:
                            print('\n# z1 dc pixels:', gf.dc_area.count())
                            print('# dc_ts_pixels:', dc_ts_pixels)
                            print('ts_dayfrac_fn:', ts_dayfrac_fn_wglacier)
                            var_full2plot = gf.ts.copy()
                            clim = (0,24)
                            plot_array(var_full2plot, clim, [glac_str + ' Ts_dayfrac'], 'inferno', 'dayfrac', 
                                       close_fig=False)
                    ts_key_count += 1
                    
                
                        
            # ===== Add layers =====
            # Record filenames if all exist
            if (ts_fn_wglacier is not None and ts_year_fn_wglacier is not None and ts_doy_fn_wglacier is not None and 
                ts_dayfrac_fn_wglacier is not None):                    
                ts_fns_df.loc[glac_idx,:] = [rgiid, ts_fn_wglacier, ts_year_fn_wglacier, ts_doy_fn_wglacier, 
                                             ts_dayfrac_fn_wglacier]
                    
    # Save updated filenames
    ts_fns_df.to_csv(debris_prms.ts_fp + debris_prms.ts_fns_fn, index=False)

20331 ex: 13.00067 

8874 glaciers in region 13 are included in this model run: ['00067', '00080', '00093', '00137', '00175', '00188', '00190', '00198', '00199', '00200', '00202', '00203', '00209', '00210', '00211', '00217', '00218', '00221', '00222', '00223', '00226', '00228', '00233', '00235', '00236', '00240', '00246', '00248', '00250', '00299', '00335', '00345', '00358', '00386', '00391', '00394', '00399', '00400', '00402', '00413', '00420', '00500', '00502', '00503', '00505', '00507', '00508', '00515', '00516', '00517'] and more
5873 glaciers in region 14 are included in this model run: ['00005', '00018', '00020', '00026', '00028', '00029', '00032', '00033', '00036', '00043', '00056', '00057', '00063', '00065', '00072', '00075', '00079', '00097', '00101', '00104', '00122', '00127', '00131', '00142', '00145', '00146', '00154', '00155', '00163', '00213', '00219', '00222', '00225', '00243', '00251', '00271', '00287', '00288', '00323', '00332', '00342', '00346', '00347', '00350', '003

/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


46 46 RGI60-13.00508
47 47 RGI60-13.00515
48 48 RGI60-13.00516
49 49 RGI60-13.00517
50 50 RGI60-13.00519
51 51 RGI60-13.00520
52 52 RGI60-13.00521
53 53 RGI60-13.00522
54 54 RGI60-13.00524
55 55 RGI60-13.00525
56 56 RGI60-13.00543
57 57 RGI60-13.00547
58 58 RGI60-13.00548
59 59 RGI60-13.00549
60 60 RGI60-13.00550
61 61 RGI60-13.00552
62 62 RGI60-13.00553
63 63 RGI60-13.00557
64 64 RGI60-13.00558
65 65 RGI60-13.00559
66 66 RGI60-13.00568
67 67 RGI60-13.00569
68 68 RGI60-13.00571
69 69 RGI60-13.00573
70 70 RGI60-13.00574
71 71 RGI60-13.00575
72 72 RGI60-13.00591
73 73 RGI60-13.00599
74 74 RGI60-13.00600
75 75 RGI60-13.00601
76 76 RGI60-13.00602
77 77 RGI60-13.00604
78 78 RGI60-13.00605
79 79 RGI60-13.00606
80 80 RGI60-13.00611
81 81 RGI60-13.00612
82 82 RGI60-13.00616
83 83 RGI60-13.00617
84 84 RGI60-13.00618
85 85 RGI60-13.00619
86 86 RGI60-13.00620
87 87 RGI60-13.00622
88 88 RGI60-13.00623
89 89 RGI60-13.00624
90 90 RGI60-13.00626
91 91 RGI60-13.00628
92 92 RGI60-13.00629
93 93 RGI60-1

407 407 RGI60-13.01300
408 408 RGI60-13.01301
409 409 RGI60-13.01302
410 410 RGI60-13.01303
411 411 RGI60-13.01306
412 412 RGI60-13.01307
413 413 RGI60-13.01308
414 414 RGI60-13.01309
415 415 RGI60-13.01311
416 416 RGI60-13.01312
417 417 RGI60-13.01316
418 418 RGI60-13.01317
419 419 RGI60-13.01318
420 420 RGI60-13.01323
421 421 RGI60-13.01325
422 422 RGI60-13.01326
423 423 RGI60-13.01328
424 424 RGI60-13.01329
425 425 RGI60-13.01332
426 426 RGI60-13.01333
427 427 RGI60-13.01334
428 428 RGI60-13.01335
429 429 RGI60-13.01336
430 430 RGI60-13.01337
431 431 RGI60-13.01338
432 432 RGI60-13.01339
433 433 RGI60-13.01341
434 434 RGI60-13.01342
435 435 RGI60-13.01343
436 436 RGI60-13.01344
437 437 RGI60-13.01346
438 438 RGI60-13.01348
439 439 RGI60-13.01350
440 440 RGI60-13.01355
441 441 RGI60-13.01356
442 442 RGI60-13.01357
443 443 RGI60-13.01358
444 444 RGI60-13.01359
445 445 RGI60-13.01360
446 446 RGI60-13.01361
447 447 RGI60-13.01363
448 448 RGI60-13.01364
449 449 RGI60-13.01365
450 450 RGI